In [127]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from pathlib import Path


from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [128]:
import zipfile

In [129]:
def unzip_folder(zip_path, dest_path):
    with zipfile.ZipFile(zip_path, "r") as zip_file:
        zip_file.extractall(dest_path)

In [130]:
zip_path = r"/Users/arnel/Downloads/Instacart Market Basket Analysis"
dest_path =r"/Users/arnel/Downloads/Instacart data"


In [131]:
# Begin looping through zipped folder 
for root, dirs, files in os.walk(zip_path):
    # Loop through files of zipped folders all at once 
    for file in files:
        if file.endswith(".zip"):
            # Create path that points directly to zipped file 
            full_file_path = os.path.join(root, file)
            # Run unzip_folder() using new full_file_path and destination
            unzip_folder(full_file_path, dest_path)

In [132]:
# Read the Instacart data into dataframes 
# Change directories to where unzipped Instacart data is located 
os.chdir(dest_path)

In [133]:
# List the files in my current working directory 
sourcefiles = os.listdir(dest_path)
sourcefiles

['products.csv',
 'orders.csv',
 '.DS_Store',
 'order_products__train.csv',
 'departments.csv',
 '__MACOSX',
 'aisles.csv',
 'order_products__prior.csv',
 'sample_submission.csv']

In [134]:
file_names = [os.path.splitext(file)[0] for file in sourcefiles if file.endswith(".csv")]

In [135]:
print(file_names)

['products', 'orders', 'order_products__train', 'departments', 'aisles', 'order_products__prior', 'sample_submission']


In [136]:
# dfs = {key: pd.read_csv(os.path.join(dest_path, file)) for key, file in zip(file_names, sourcefiles) if file.endswith(".csv")}
dfs = {file.stem: pd.read_csv(file.resolve()) for file in Path(dest_path).glob("*.csv")}

In [137]:
dfs.keys()

dict_keys(['products', 'orders', 'order_products__train', 'departments', 'aisles', 'order_products__prior', 'sample_submission'])

In [138]:
products_df = dfs["products"]
orders_df = dfs["orders"]
departments_df = dfs["departments"]
aisles_df = dfs["aisles"]
order_products_prior_df = dfs["order_products__prior"]

In [156]:
products_df.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id'], dtype='object')

In [142]:
print(f'The dimension for the Dataframes are as follows:',
'\n''Products:',products_df.shape, 
'\n''Orders:', orders_df.shape,
'\n''Department:', departments_df.shape,
'\n''Aisles:', aisles_df.shape,
'\n''Prior Orders:', order_products_prior_df.shape)

The dimension for the Dataframes are as follows: 
Products: (49688, 4) 
Orders: (3421083, 7) 
Department: (21, 2) 
Aisles: (134, 2) 
Prior Orders: (32434489, 4)


In [148]:
# Merge the aisles dataframe with the products dataframe to create aisle_prodcut_merged 
# merged on 'order_id' using a 'inner' join
aisle_product_merged = pd.merge(aisles_df, products_df, on = 'aisle_id', how = 'inner')
aisle_product_merged

,aisle_id,aisle,product_id,product_name,department_id
0,1,prepared soups salads,209,Italian Pasta Salad,20
1,1,prepared soups salads,554,Turkey Chili,20
2,1,prepared soups salads,886,Whole Grain Salad with Roasted Pecans & Mango ...,20
3,1,prepared soups salads,1600,Mediterranean Orzo Salad,20
4,1,prepared soups salads,2539,Original Potato Salad,20
...,...,...,...,...,...
49683,134,specialty wines champagnes,47713,Sparkling Rose,5
49684,134,specialty wines champagnes,48984,Dry Sherry,5
49685,134,specialty wines champagnes,49438,Dry Fino Sherry,5
49686,134,specialty wines champagnes,49562,Blanc De Noirs Sparkling Wine,5


In [149]:
print(f'The shape of the new data frame', aisle_product_merged.shape)

The shape of the new data frame (49688, 5)


In [150]:
department_aisle_merged = pd.merge(aisle_product_merged, departments_df, on = "department_id", how = "inner")
department_aisle_merged

,aisle_id,aisle,product_id,product_name,department_id,department
0,1,prepared soups salads,209,Italian Pasta Salad,20,deli
1,1,prepared soups salads,554,Turkey Chili,20,deli
2,1,prepared soups salads,886,Whole Grain Salad with Roasted Pecans & Mango ...,20,deli
3,1,prepared soups salads,1600,Mediterranean Orzo Salad,20,deli
4,1,prepared soups salads,2539,Original Potato Salad,20,deli
...,...,...,...,...,...,...
49683,100,missing,49553,Organic Plain Skyr,21,missing
49684,100,missing,49575,Chocolate Vanilla Malt Custard,21,missing
49685,100,missing,49641,"8\"" Pecan Pie",21,missing
49686,100,missing,49664,Lemon Cayenne Drinking Vinegar,21,missing


In [151]:
print(f'The shape of the new data frame', department_aisle_merged.shape)

The shape of the new data frame (49688, 6)


In [162]:
order_product_merged = pd.merge(orders_df,order_products_prior_df , on = 'order_id', how = 'inner')
order_product_merged.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0
5,2398795,1,prior,2,3,7,15.0,196,1,1
6,2398795,1,prior,2,3,7,15.0,10258,2,0
7,2398795,1,prior,2,3,7,15.0,12427,3,1
8,2398795,1,prior,2,3,7,15.0,13176,4,0
9,2398795,1,prior,2,3,7,15.0,26088,5,1


In [166]:
prior_orders_product_merged = pd.merge(order_products_prior_df, department_aisle_merged, on = 'product_id', how = 'inner')
prior_orders_product_merged.head(10)


,order_id,product_id,add_to_cart_order,reordered,aisle_id,aisle,product_name,department_id,department
0,2,33120,1,1,86,eggs,Organic Egg Whites,16,dairy eggs
1,26,33120,5,0,86,eggs,Organic Egg Whites,16,dairy eggs
2,120,33120,13,0,86,eggs,Organic Egg Whites,16,dairy eggs
3,327,33120,5,1,86,eggs,Organic Egg Whites,16,dairy eggs
4,390,33120,28,1,86,eggs,Organic Egg Whites,16,dairy eggs
5,537,33120,2,1,86,eggs,Organic Egg Whites,16,dairy eggs
6,582,33120,7,1,86,eggs,Organic Egg Whites,16,dairy eggs
7,608,33120,5,1,86,eggs,Organic Egg Whites,16,dairy eggs
8,623,33120,1,1,86,eggs,Organic Egg Whites,16,dairy eggs
9,689,33120,4,1,86,eggs,Organic Egg Whites,16,dairy eggs


In [165]:
prior_orders_product_merged.shape

(32434489, 9)

In [170]:
#most_ordered = prior_orders_product_merged.groupby("product_name").agg(count_items = pd.NamedAgg("product_name", "count_item"))
#most_orderd_10 =most_ordered .head(10)                                                                                              

In [ ]:
bar_plot = go.Bar(
    x = 